# BGE-M3 Korean Embedding Fine-tuning (Azure ML Studio)

이 노트북은 합성 데이터 생성 → 데이터 자산 등록 → 실험/실행 제출 → 모델 등록 → 엔드포인트 배포를 단계적으로 수행합니다.

필수 환경 변수:
- AZURE_SUBSCRIPTION_ID
- AZURE_RESOURCE_GROUP
- AZUREML_WORKSPACE_NAME
- AZUREML_COMPUTE_NAME (예: cpu-cluster)

In [ ]:
from pathlib import Path
import os

from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.entities import Data, AmlCompute, ManagedOnlineEndpoint, ManagedOnlineDeployment, Model
from azure.identity import DefaultAzureCredential

from src.synth_data import read_terms, synthesize_examples, save_jsonl

subscription_id = os.environ.get('AZURE_SUBSCRIPTION_ID')
resource_group = os.environ.get('AZURE_RESOURCE_GROUP')
workspace_name = os.environ.get('AZUREML_WORKSPACE_NAME')
compute_name = os.environ.get('AZUREML_COMPUTE_NAME', 'cpu-cluster')

assert subscription_id and resource_group and workspace_name, '환경 변수를 확인하세요.'

## 1) 합성 데이터 생성

In [ ]:
terms = read_terms(Path('data/terms_electronics_ko.txt'))
records = synthesize_examples(terms, n_pairs_per_term=6, seed=42)
output_path = Path('data/synthetic_electronics.jsonl')
save_jsonl(records, output_path)
len(records)

## 2) Azure ML Client 생성

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## 3) 데이터 자산 등록

In [ ]:
data_asset = Data(
    name='bge-m3-kr-synth-train',
    version='1',
    type='uri_file',
    path=str(output_path),
)
data_asset = ml_client.data.create_or_update(data_asset)
data_asset

## 4) 컴퓨트 클러스터 준비

In [ ]:
try:
    ml_client.compute.get(compute_name)
    print('Compute exists:', compute_name)
except Exception:
    compute = AmlCompute(name=compute_name, size='Standard_DS3_v2', min_instances=0, max_instances=2)
    ml_client.compute.begin_create_or_update(compute).result()
    print('Compute created:', compute_name)

## 5) 실험(Experiment) 및 실행(Run) 제출

In [ ]:
job = command(
    name='bge-m3-kr-finetune',
    experiment_name='bge-m3-kr-finetune',
    code='.',
    command='python -m src.train --train_data ${{inputs.train_data}} --output_dir ${{outputs.model_output}} --epochs 1 --batch_size 16 --lr 2e-5 --max_seq_length 256',
    inputs={
        'train_data': Input(type='uri_file', path=data_asset.id),
    },
    outputs={
        'model_output': {'type': 'uri_folder'},
    },
    environment='AzureML-pytorch-2.1-ubuntu20.04-py310-cpu@latest',
    compute=compute_name,
)
created_job = ml_client.jobs.create_or_update(job)
created_job

In [ ]:
ml_client.jobs.stream(created_job.name)

## 6) 모델 등록

In [ ]:
model = Model(
    name='bge-m3-kr-embedding-model',
    path=created_job.outputs['model_output'],
    type='custom_model',
)
registered_model = ml_client.models.create_or_update(model)
registered_model

## 7) 엔드포인트 및 배포

In [ ]:
endpoint = ManagedOnlineEndpoint(
    name='bge-m3-kr-embeddings',
    auth_mode='key',
)
ml_client.begin_create_or_update(endpoint).result()

deployment = ManagedOnlineDeployment(
    name='blue',
    endpoint_name=endpoint.name,
    model=registered_model.id,
    environment='AzureML-pytorch-2.1-ubuntu20.04-py310-cpu@latest',
    instance_type='Standard_DS3_v2',
    instance_count=1,
    code_configuration={
        'code': '.',
        'scoring_script': 'src/score.py',
    },
)
ml_client.begin_create_or_update(deployment).result()

ml_client.online_endpoints.begin_traffic_update(
    name=endpoint.name, traffic={'blue': 100}
).result()

## 8) 엔드포인트 테스트

배포된 엔드포인트에 샘플 요청을 보내 정상 작동을 확인합니다.

In [ ]:
import json

# 배포 디렉토리 생성
deploy_dir = Path('deploy')
deploy_dir.mkdir(exist_ok=True)

# 샘플 요청 데이터 생성 (임베딩 모델용)
sample_request = {
    "inputs": {
        "data": [
            "전자제품 추천 부탁드립니다",
            "노트북 구매 고려중입니다",
            "스마트폰 배터리 수명이 궁금합니다"
        ]
    }
}

# JSON 파일로 저장
sample_request_path = deploy_dir / 'sample-request.json'
with open(sample_request_path, 'w', encoding='utf-8') as f:
    json.dump(sample_request, f, ensure_ascii=False, indent=2)

print(f"샘플 요청 파일 생성: {sample_request_path}")
print(json.dumps(sample_request, ensure_ascii=False, indent=2))

In [ ]:
# 엔드포인트 호출 테스트
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint.name,
    request_file=str(sample_request_path),
    deployment_name='blue'
)

print("응답 결과:")
print(response)

## 9) 리소스 정리

비용 절감을 위해 사용하지 않는 리소스를 정리합니다.

In [ ]:
# 엔드포인트 삭제 (필요시 주석 해제)
# ml_client.online_endpoints.begin_delete(name=endpoint.name).result()
# print(f"엔드포인트 '{endpoint.name}' 삭제 완료")

# 컴퓨팅 클러스터는 자동으로 스케일 다운되므로 별도 삭제 불필요
# 하지만 완전히 삭제하려면:
# ml_client.compute.begin_delete(compute_name).result()
# print(f"컴퓨팅 클러스터 '{compute_name}' 삭제 완료")